<a href="https://colab.research.google.com/github/sheriefpa/Myworks/blob/main/covid_19_vaccine_tweets_sentiment_analysis_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center><span style="font-family:cursive;"> 🦠Covid-19 Vaccine tweets sentiment analysis🙂😐🙁 </span></center>



<center><img src="https://static.toiimg.com/thumb/77928564.cms?width=680&height=512&imgsize=126320" width="600";  height:"100"></center>


In [10]:
import warnings
warnings.filterwarnings('ignore')


In [9]:
import numpy as np
import spacy
import torchtext
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.legacy import data
import torch.nn.functional as F
import gc


import matplotlib.pyplot as plt
import glob
import plotly.express as px
import warnings
import plotly as py
import plotly.graph_objs as go

import string
import re


In [1]:
!pip install torchtext==0.9.1

     |████████████████████████████████| 7.1 MB 24.4 MB/s 
     |████████████████████████████████| 804.1 MB 2.2 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.12.0
    Uninstalling torchtext-0.12.0:
      Successfully uninstalled torchtext-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.8.1 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.8.1 which is incompatible.


Tokenization using Spacy

In [2]:
!pip install spacy
import spacy

In [3]:
nlp = spacy.load(name = 'en_core_web_sm')

In [4]:
!python -m spacy download en_core_web_md
import en_core_web_md
nlp_1 = en_core_web_md.load()

     |████████████████████████████████| 96.4 MB 1.2 MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-py3-none-any.whl size=98051301 sha256=9a69471ba4e54d97105d41f9cbaf2fa7e9028d3a1afa01417be2d149065a1550
  Stored in directory: /tmp/pip-ephem-wheel-cache-r35d1424/wheels/69/c5/b8/4f1c029d89238734311b3269762ab2ee325a42da2ce8edb997
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [11]:
!pip install emoji



     |████████████████████████████████| 175 kB 26.3 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=55dc0517f17cf57fd1e0ae914b146a372a5a784170ff05f7b799e0948ab7735d
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [16]:
import en_core_web_sm

nlp = en_core_web_sm.load()

In [12]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My Drive/Sentiment Analysis


Mounted at /content/drive
/content/drive/My Drive/Sentiment Analysis


In [13]:
tweetsDF = pd.read_csv("covid-19_vaccine_tweets_with_sentiment.csv", encoding = "ISO-8859-1")

In [14]:
tweetsDF

,tweet_id,label,tweet_text
0,1.360342e+18,1,"4,000 a day dying from the so called Covid-19 ..."
1,1.382896e+18,2,Pranam message for today manifested in Dhyan b...
2,1.375673e+18,2,Hyderabad-based ?@BharatBiotech? has sought fu...
3,1.381311e+18,1,"Confirmation that Chinese #vaccines ""dont hav..."
4,1.362166e+18,3,"Lab studies suggest #Pfizer, #Moderna vaccines..."
...,...,...,...
5995,1.370975e+18,2,@Swamy39 Dr. @Swamy39 jee :\n\nMany people lik...
5996,1.379827e+18,3,So happy to be fully vaccinated against COVID-...
5997,1.384789e+18,2,Serum Institute of India announces cost of Cov...
5998,1.382355e+18,1,@___batshitcrazy @BarrowfordHead @Bectully I h...


In [20]:
#tweetsDF[doc1] = nlp(tweetsDF.tweet_text[])
tweetsDF['doc1'] = tweetsDF['tweet_text'].apply(nlp)

In [26]:
#!wget kaggle kernels output afiaibnath/covid-19-vaccine-tweets-sentiment-analysis -p /path/to/dest


In [31]:
tweetsDF.head()

,tweet_id,label,tweet_text
0,1.360342e+18,1,"4,000 a day dying from the so called Covid-19 ..."
1,1.382896e+18,2,Pranam message for today manifested in Dhyan b...
2,1.375673e+18,2,Hyderabad-based ?@BharatBiotech? has sought fu...
3,1.381311e+18,1,"Confirmation that Chinese #vaccines ""dont hav..."
4,1.362166e+18,3,"Lab studies suggest #Pfizer, #Moderna vaccines..."


In [32]:
tweetsDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tweet_id    6000 non-null   float64
 1   label       6000 non-null   int64  
 2   tweet_text  6000 non-null   object 
dtypes: float64(1), int64(1), object(1)
memory usage: 140.8+ KB


In [33]:
labels=tweetsDF.groupby("label").agg({'tweet_text':'count'}).rename(columns={'tweet_text':'tweet_count'}).sort_values(by="tweet_count", ascending=False)

In [ ]:
labels

,tweet_count
label,
2,3680
3,1900
1,420


# <center><span style="font-family:cursive;"> There are 3 types of sentiment labels: Negative = 1, neutral = 2, positive = 3  </span></center>
Let's visualize frequency of each label

In [34]:
fig1 = px.bar(labels, x="tweet_count", y=labels.index.astype('str'), orientation='h', hover_data=['tweet_count'],labels={'y':'Labels'}, title='Label Counts')
fig1.update_layout(yaxis={'categoryorder':'total ascending'})
fig1.show()

# <center><span style="font-family:cursive;"> Raw tweet text need some preprocessing.Below functions are used for various preprocessing. </span></center>

In [39]:
def removeStopwords(text):
    doc = nlp(text)
    clean_text = ' '
    for txt in doc:
        if (txt.is_stop == False):
            clean_text = clean_text + " " + str(txt)        
    return clean_text

def removePunctuations(text):
    return text.translate(str.maketrans('', '', string.punctuation)).replace(" s ", " ")

def removeLinks(text):
    clean_text = re.sub('https?://\S+|www\.\S+', '', text)
    return clean_text

def removeNumbers(text):
    clean_text = re.sub(r'\d+', '', text)
    return clean_text
def removenewline(text):
    clean_text = re.sub(r'(\n+)', '', text)
    return clean_text


import emoji
#does the text contain an emoji?
def text_has_emoji(text):
    for character in text:
        if character in emoji.UNICODE_EMOJI:
            return True
    return False
#remove the emoji
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

In [40]:
def clean(text):
    if text_has_emoji(text):
        text=deEmojify(text)
    text = text.lower()
    text =removenewline(text)
    text = removeStopwords(text)
    text = removePunctuations(text)
    text = removeNumbers(text)
    text = removeLinks(text)
    return text

In [41]:
tweetsDF['text_clean'] = tweetsDF['tweet_text'].apply(clean)
tweetsDF['label'] = tweetsDF['label'].map({1:0, 2:1, 3:2})# renumbering labels to avoid error in the one hot encoding process
tweetsDF.drop("tweet_text",axis=1).to_csv('mycsvfile.csv',index=False)

In [42]:
tweetsDF.head()

,tweet_id,label,tweet_text,text_clean
0,1.360342e+18,0,"4,000 a day dying from the so called Covid-19 ...",day dying called covid vaccine dailybeast...
1,1.382896e+18,1,Pranam message for today manifested in Dhyan b...,pranam message today manifested dhyan meenap...
2,1.375673e+18,1,Hyderabad-based ?@BharatBiotech? has sought fu...,hyderabad based bharatbiotech sought fund...
3,1.381311e+18,0,"Confirmation that Chinese #vaccines ""dont hav...",confirmation chinese vaccines dont high p...
4,1.362166e+18,2,"Lab studies suggest #Pfizer, #Moderna vaccines...",lab studies suggest pfizer moderna vaccin...


In [43]:

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                tokenizer_language = 'en_core_web_sm',
                 lower = True)

LABEL = data.LabelField(dtype=torch.int64)

In [44]:
fields = [("tweet_id",None), ('label',LABEL),('text_clean', TEXT)]

In [45]:
twitterDataset = data.dataset.TabularDataset(
        path="./mycsvfile.csv", 
        format="CSV", 
        fields=fields,
        #csv_reader_params={"quotechar": None,"delimiter":";"},
        skip_header=True)

In [46]:
(train, test, valid) = twitterDataset.split(split_ratio=[0.6,0.2,0.2],
                                            stratified=True, strata_field='label')

(len(train),len(test),len(valid))

(3600, 1200, 1200)

In [47]:
MAX_VOCAB_SIZE = 14309

TEXT.build_vocab(train, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train)

In [48]:
TEXT.vocab.freqs.most_common(10)

[(' ', 17759),
 ('  ', 5156),
 ('vaccine', 1796),
 ('   ', 1511),
 ('covid', 1449),
 ('moderna', 1359),
 ('covaxin', 773),
 ('sputnikv', 597),
 ('covidvaccine', 486),
 ('pfizer', 470)]

In [49]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 14311
Unique tokens in LABEL vocabulary: 3


In [50]:
print(TEXT.vocab.freqs.most_common(20))

[(' ', 17759), ('  ', 5156), ('vaccine', 1796), ('   ', 1511), ('covid', 1449), ('moderna', 1359), ('covaxin', 773), ('sputnikv', 597), ('covidvaccine', 486), ('pfizer', 470), ('dose', 417), ('vaccines', 412), ('pfizerbiontech', 403), ('amp', 338), ('got', 317), ('    ', 299), ('vaccinated', 298), ('shot', 287), ('vaccination', 286), ('doses', 264)]


In [51]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', ' ', '  ', 'vaccine', '   ', 'covid', 'moderna', 'covaxin', 'sputnikv']


In [52]:
print(LABEL.vocab.stoi)

defaultdict(None, {'1': 0, '2': 1, '0': 2})


In [53]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test), 
    batch_size = BATCH_SIZE,
    sort_within_batch=False,
    sort_key = lambda x: len(x.text_clean),
    device = device)

# <center><span style="font-family:cursive;"> Simple 2 layer LSTM model to train </span></center>

In [54]:
import torch.nn as nn

class SentimentRNN(nn.Module):
    
    def __init__(self,corpus_size,output_size,embedd_dim,hidden_dim,n_layers):
        super().__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(corpus_size,embedd_dim)
        self.lstm = nn.LSTM(embedd_dim, hidden_dim,n_layers,dropout=0.5, batch_first=True)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_dim,output_size)
        self.act = nn.Sigmoid()
        
    def forward(self,x,hidden):
        batch_size = x.size(0)
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds,hidden)
        lstm_out = lstm_out.contiguous().view(-1,self.hidden_dim)
        out = self.dropout(lstm_out)
        out = self.fc(out)
        out = self.act(out)
        out = out.view(batch_size,-1)
        out = out[:,-3:]
        return out
    def init_hidden(self,batch_size):
        
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                   weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        #print(weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().shape ,weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().shape )
        return hidden

In [55]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 3
n_layers = 2
model  = SentimentRNN(INPUT_DIM, OUTPUT_DIM,  EMBEDDING_DIM, HIDDEN_DIM,n_layers)
#model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [56]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,324,799 trainable parameters


In [ ]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
#criterion = nn.CrossEntropyLoss()

#criterion =nn.NLLLoss()
def criterion(input, target, size_average=True):
    """Categorical cross-entropy with logits input and one-hot target"""
    l = -(target * torch.log(F.softmax(input, dim=1) + 1e-10)).sum(1)
    if size_average:
        l = l.mean()
    else:
        l = l.sum()
    return l

# <center><span style="font-family:cursive;"> Training model and checking accuracy and loss</span></center>

In [ ]:

#print(h)
def traindef(epochs, model, optimizer, criterion, train_iterator, valid_iterator):
    for epoch in range(1, epochs+1):
     
        training_loss = 0.0
        valid_loss = 0.0
        train_acc=0.0
        valid_acc = 0.0
        model.train()
        
        for batch_idx , batch in enumerate(train_iterator):
            gc.collect()
            h=model.init_hidden(BATCH_SIZE)
            h = tuple([each.data for each in h])
            if batch.text_clean.shape[1] != BATCH_SIZE:
                break
            optimizer.zero_grad()
            predict = model(batch.text_clean.permute(1,0) ,h)
            labels=torch.nn.functional.one_hot(batch.label, num_classes=3)
            loss = criterion(predict,labels)
            loss.backward()
            optimizer.step()
            training_loss += loss.cpu().detach().numpy()
            train_acc += (predict.argmax(dim=1) == labels.argmax(dim=1)).float().mean()
        training_loss /= len(train_iterator)
        train_acc/= len(train_iterator)
 
        best_valid_loss = float('inf') 
        
        model.eval()
        for batch_idx,batch in enumerate(valid_iterator): 
            h=model.init_hidden(BATCH_SIZE)
            h = tuple([each.data for each in h])
            if batch.text_clean.shape[1] != BATCH_SIZE:
                break
            predict = model(batch.text_clean.permute(1,0),h)
            labels=torch.nn.functional.one_hot(batch.label, num_classes=3)
            loss = criterion(predict,labels)
            valid_loss += loss.cpu().detach().numpy()
            valid_acc += (predict.argmax(dim=1) == labels.argmax(dim=1)).float().mean()
 
        valid_loss /= len(valid_iterator)
        valid_acc/= len(valid_iterator)
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), 'tut1-model.pt')
        #print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}'.format(epoch, training_loss, valid_loss))
        print(f'Epoch: {epoch}')
        print(f'\tTrain Loss: {training_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

In [ ]:
traindef(5, model, optimizer, criterion, train_iterator, valid_iterator)

Epoch: 1
	Train Loss: 0.535 | Train Acc: 30.40%
	 Val. Loss: 1.033 |  Val. Acc: 58.06%
Epoch: 2
	Train Loss: 0.267 | Train Acc: 15.27%
	 Val. Loss: 1.033 |  Val. Acc: 58.06%
Epoch: 3
	Train Loss: 0.877 | Train Acc: 50.14%
	 Val. Loss: 1.032 |  Val. Acc: 58.06%
Epoch: 4
	Train Loss: 0.515 | Train Acc: 29.19%
	 Val. Loss: 1.032 |  Val. Acc: 58.06%
Epoch: 5
	Train Loss: 0.591 | Train Acc: 32.98%
	 Val. Loss: 1.032 |  Val. Acc: 58.06%


In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))

<All keys matched successfully>

In [ ]:
model.eval()
test_loss=0.0
test_acc=0.0
for batch_idx,batch in enumerate(test_iterator): 
    h=model.init_hidden(BATCH_SIZE)
    h = tuple([each.data for each in h])
    if batch.text_clean.shape[1] != BATCH_SIZE:
        break
    predict = model(batch.text_clean.permute(1,0),h)
    labels=torch.nn.functional.one_hot(batch.label, num_classes=3)
    loss = criterion(predict,labels)
    test_loss += loss.cpu().detach().numpy()
    test_acc += (predict.argmax(dim=1) == labels.argmax(dim=1)).float().mean()

test_loss /= len(test_iterator)
test_acc/= len(test_iterator)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 1.032 | Test Acc: 57.89%
